In [1]:
import sys, os

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

import muspy
from pathlib import Path
from src.utils import sequence_extraction

In [2]:
song = muspy.read('../midi/examples/bitmidi/pink-panther.mid')
song

Music(metadata=Metadata(schema_version='0.0', source_filename='pink-panther.mid', source_format='midi'), resolution=240, tempos=[Tempo(time=0, qpm=125.0)], time_signatures=[TimeSignature(time=0, numerator=4, denominator=4)], tracks=[Track(program=0, is_drum=False, name='Melody', notes=[Note(time=640, pitch=56, duration=80, velocity=100), Note(time=720, pitch=57, duration=80, velocity=100), Note(time=880, pitch=58, duration=80, velocity=100), ...]), Track(program=0, is_drum=False, name='Piano Accomp', notes=[Note(time=640, pitch=37, duration=80, velocity=100), Note(time=640, pitch=44, duration=80, velocity=100), Note(time=640, pitch=49, duration=80, velocity=100), ...]), Track(program=0, is_drum=False, name='Alto Sax', notes=[Note(time=46960, pitch=51, duration=77, velocity=112), Note(time=47040, pitch=52, duration=77, velocity=112), Note(time=47440, pitch=54, duration=77, velocity=112), ...]), ...])

In [5]:
success = sequence_extraction.extract_melodies_to_folder(song, 'pink_panther', 'test_set', 4)
if success == 1:
    print("Sequences successfully extracted.")